In [5]:
import numpy as np

In [480]:
class FloatTensor(object):
    
    def __init__(self,data,autograd=False,keepgrads=False,creator=None, creation_op=None, id=None):
        
        self.data = data
        self.autograd = autograd
        self.keepgrads = keepgrads
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id
        
        self.creator = creator
        self.creation_op = creation_op
        self.children = {}
    
    def __add__(self,other):
        if(self.autograd):
            out = FloatTensor(self.data + other.data, autograd=True, creator=[self,other], creation_op="add")
            self.children[out.id] = 0
            other.children[out.id] = 0
            return out
        else:
            return FloatTensor(self.data + other.data)
        
    
    def __mul__(self,other):
        if(self.autograd):
            out = FloatTensor(self.data * other.data, autograd=True, creator=[self,other], creation_op="mul")
            self.children[out.id] = 0
            other.children[out.id] = 0
            
            return out
        else:
            return FloatTensor(self.data * other.data)

    def __truediv__(self,other):
        if(self.autograd):
            out = FloatTensor(self.data / other.data, autograd=True, creator=[self,other], creation_op="div")
            self.children[out.id] = 0
            other.children[out.id] = 0
            
        return FloatTensor(self.data / other.data)    
    
    def __sub__(self,other):
        if(self.autograd):
            out = FloatTensor(self.data - other.data, autograd=True, creator=[self,other], creation_op="sub")
            self.children[out.id] = 0
            other.children[out.id] = 0
            
            return out

        return FloatTensor(self.data - other.data)   
    
    def __repr__(self):
        return self.data.__repr__()
    
    def __neg__(self):
        if(self.autograd):
            out = FloatTensor(-self.data, autograd=True, creator=[self], creation_op="neg")
            self.children[out.id] = 0
        return FloatTensor(-self.data)      
    
    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt == 0):
                return False
        return True
    
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
            if(grad is None):
                grad = FloatTensor(np.ones_like(self.data))
            
            if(grad_origin is not None):
                if(self.children[grad_origin.id] > 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] += 1
            
            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
            
            # grads must not have grads of their own
            assert grad.autograd == False
            
            # only continue backpropping if there's something to backprop into
            # only continue backpropping if all gradients (from children) are accounted for
            # override waiting for children if "backprop" was called on this variable directly
            if(self.creator is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):

                if(self.creation_op == "add"):
                    self.creator[0].backward(grad, self)
                    self.creator[1].backward(grad, self)

                if(self.creation_op == "mul"):
                    self.creator[0].backward(self.grad * self.creator[1], self)
                    self.creator[1].backward(self.grad * self.creator[0], self)

                if(self.creation_op == "div"):
                    self.creator[0].backward(self.grad / self.creator[1], self)
                    self.creator[1].backward(self.grad / self.creator[0], self)

                if(self.creation_op == "sub"):
                    self.creator[0].backward(self.grad, self)
                    self.creator[1].backward(-self.grad, self)

                if(self.creation_op == "neg"):
                    self.creator[0].backward(-self.grad,self)
                    
                if(not self.keepgrads):
                    self.grad = None

        
    

In [503]:
a = FloatTensor(np.array([1,2,3,4,5]),autograd=True,keepgrads=True)
b = FloatTensor(np.array([6,7,8,9,10]),autograd=True,keepgrads=True)

In [504]:
c = a - b
c.id

89874

In [505]:
d = a * b
d.id

4810

In [506]:
e = c + d
e.id

57150

In [507]:
e.backward()

In [508]:
a.grad

array([ 7,  8,  9, 10, 11])

In [509]:
b.grad

array([0, 1, 2, 3, 4])

In [510]:
import torch
from torch.autograd import Variable

In [511]:
a = Variable(torch.FloatTensor(np.array([1.,2,3,4,5])),requires_grad=True)
b = Variable(torch.FloatTensor(np.array([6.,7,8,9,10])),requires_grad=True)

In [512]:
c = a - b

In [513]:
d = a * b

In [514]:
e = c + d

In [515]:
e.backward(torch.FloatTensor(np.ones(5)))

In [516]:
a.grad

Variable containing:
  7
  8
  9
 10
 11
[torch.FloatTensor of size 5]

In [499]:
b.grad

Variable containing:
 0
 1
 2
 3
 4
[torch.FloatTensor of size 5]

In [500]:
c.grad

In [501]:
d.grad

In [502]:
e.grad